In [5]:
import pandas as pd
import numpy as np

from sklearn.metrics import SCORERS
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [6]:
train = pd.read_csv('../test_and_train_installs/train_data_encoded.csv')
test = pd.read_csv('../test_and_train_installs/test_data_encoded.csv')

ref_hashes = train['ref_hash']
del train['ref_hash']
del test['ref_hash']

In [7]:
Xtrain, ytrain = train.iloc[:,:-1],train.iloc[:,-1]

Xtest, ytest = test.iloc[:,:-1],test.iloc[:,-1]

In [8]:
Xtrain.fillna(0, inplace=True)
Xtest.fillna(0, inplace=True)

/home/santiago/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4034: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


In [9]:
# Perform Grid-Search
gsc = GridSearchCV(
    estimator=RandomForestRegressor(),
    param_grid={
        'max_depth': range(3,7),
        'n_estimators': (10, 50, 100, 1000),
    },
    cv=5, scoring='neg_mean_squared_error', verbose=0, n_jobs=-1)
    
grid_result = gsc.fit(Xtrain, ytrain)
best_params = grid_result.best_params_
    
rfr = RandomForestRegressor(max_depth=best_params["max_depth"], n_estimators=best_params["n_estimators"], 
                            random_state=False, verbose=False)

# Perform K-Fold CV
#scores = cross_val_score(rfr, Xtest, ytest, cv=10, scoring='mean_squared_error')
rfr.fit(Xtrain, ytrain)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=4,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=None,
           oob_score=False, random_state=False, verbose=False,
           warm_start=False)

In [10]:
preds = rfr.predict(Xtest)

In [11]:
preds

array([259200.        , 139375.92054839, 259200.        , ...,
       259200.        , 259200.        , 133889.52863945])

In [12]:
# Error RMSE
rmse = np.sqrt(mean_squared_error(ytest, preds))
print("RMSE: %f" % (rmse))

RMSE: 42778.617019


In [13]:
df = pd.DataFrame(ref_hashes)
df['predictions'] = preds
df

,ref_hash,predictions
0,1000169251625791246_sc,259200.000000
1,1000395625957344683_sc,139375.920548
2,1003027494996471685_sc,259200.000000
3,1006670001679961544_sc,259200.000000
4,1007573308966476713_sc,107906.729280
5,1010070503877148763_sc,259200.000000
6,1010265377387765028_sc,127466.247326
7,1010531372912327058_sc,125044.022820
8,1011610998357271358_sc,259200.000000
9,1013543838965040946_sc,259200.000000


In [14]:
df.to_csv('../submit_kaggle/pred_installs.csv', index=False)